# Лабораторная 1
## Подраздел: QR метод

* Cтудент: Ефимов А.В.
* Группа: М8О-307Б
* Вариант: 7

## Задание

Реализовать алгоритм QR - разложения матриц в виде программы. На его основе разработать программу, 
реализующую QR - алгоритм решения полной проблемы собственных значений произвольных матриц, 
задавая в качестве входных данных матрицу и точность вычислений. 
С использованием разработанного программного обеспечения найти собственные значения матрицы. 

## Решение

Загрузка библиотек, настройка лог сообщений и загрузка матрицы:

In [1]:
import math
import numpy as np
import logging, json
from math  import sqrt
from cmath import sqrt as comp_sqrt
from functools import reduce

# Configurations
np.set_printoptions(suppress=True)
logging.basicConfig(level=logging.DEBUG)

# Load matrix and eps
with open("task.json", "r") as json_file:
    task = json.load(json_file)

matrix = np.array(task["matrix"])
try:
    eps = task["epsilon"]
except KeyError:
    eps = 10e-5

print(f"Input matrix:\n{matrix}")
print(f"Selected epsilon: {eps}")

Input matrix:
[[ 9  0  2]
 [-6  4  4]
 [-2 -7  5]]
Selected epsilon: 0.0001


Нахождения матрицы Хаусхолдера:

In [2]:
def norm(array):
    return sqrt(reduce(lambda x, n: x + n*n, array, 0))

def calc_householder_vector(matrix, iter_no):
    matrix_size = len(matrix)
    vector = np.zeros((matrix_size, ))
    vector[iter_no:] = matrix[iter_no:, iter_no]

    vector[iter_no] += np.sign(vector[iter_no]) * norm(vector)

    return vector

def calc_householder_matrix(matrix, iter_no):
    matrix_size = len(matrix)
    identity    = np.identity(matrix_size)
    house_vec   = calc_householder_vector(matrix, iter_no)
    
    coef = 2 / np.dot(house_vec, house_vec)

    return identity - coef * (np.outer(house_vec, house_vec))

QR разложение матрицы:

In [3]:
def qr_decomposition(matrix):
    Q = 1

    for i in range(len(matrix)):
        house = calc_householder_matrix(matrix, i)
        Q = np.dot(Q, house)
        matrix = np.dot(house, matrix)

    return Q, matrix

Проверка первого условия - все элементы под первой побочной диагональю
должны быть близки к нулю:

In [4]:
def test_second_eps(matrix, eps):
    # Interested in elements below second diagonal:
    #  a00   a01   a02  a03 
    #  a10   a11   a12  a13 
    # [a20]  a21   a22  a23 
    # [a30] [a31]  a32  a33 
    # ...
    
    for i, col in enumerate(abs(matrix.T)):
        if not all(col[i + 2:] < eps):
            return False

    return True

Получение собственных значений из матрицы.
Если первое значение под диагональю близко к нулю, то на диагонали 
действительное собственное значение.
Иначе, если первое значение под диагональю не близко к нулю, 
то возможно это комплексное значение и блок образует
уравнение вида $a\lambda^2 + b\lambda + c = 0$.
Иначе алгоритм не достаточно сошелся.

In [5]:
def get_eigenval(matrix, col_i, eps):
    def square_subarray(i):
        return matrix[i:i+2, i:i+2]

    def check_complex(matrix):
        if matrix.shape != (2, 2):
            raise ValueError("Matrix should be square")

        m = matrix

        # a is 1
        b = -m[0, 0] - m[1, 1]
        c = m[0, 0] * m[1, 1] - m[1, 0] * m[0, 1]

        d = b * b - 4 * c

        return d < 0

    def complex_eigen(matrix):
        m = matrix

        b = -m[0, 0] - m[1, 1]
        c = m[0, 0] * m[1, 1] - m[1, 0] * m[0, 1]

        d = b * b - 4 * c

        sqrt_d = comp_sqrt(d)

        return ( (b + sqrt_d) / 2, (b - sqrt_d) / 2 )

    if norm(abs(matrix[col_i+1:, col_i])) <= eps:
        return (True, matrix[col_i, col_i], 0)

    elif check_complex(square_subarray(col_i)):
        prev_comp_0, prev_comp_1 = complex_eigen(square_subarray(col_i))
        
        Q, R = qr_decomposition(matrix)
        matrix = R @ Q

        comp_0, comp_1 = complex_eigen(square_subarray(col_i))

        close = [0, 0]
        close[0] = abs(comp_0 - prev_comp_0) <= eps
        close[1] = abs(comp_1 - prev_comp_1) <= eps

        if all(close):
            return (True, comp_0, comp_1)

    return (False, 0, 0)


Сам QR-алгоритм:

In [6]:
def qr_algorithm(matrix, eps):
    matrix_size = len(matrix)

    while not test_second_eps(matrix, eps):
        Q, R = qr_decomposition(matrix)
        matrix = R @ Q

    logging.debug("Reduced matrix:\n%s", str(matrix))

    col_i = 0
    values = np.zeros((matrix_size, ), dtype=np.complex64)

    while col_i < matrix_size - 1:
        exists, val_1, val_2 = get_eigenval(matrix, col_i, eps)
        if exists:
            if np.iscomplex(val_1):
                logging.debug("Complex values found: %s %s", str(val_1), str(val_2))
                values[col_i] = val_1
                values[col_i + 1] = val_2
                col_i += 2
            else:
                logging.debug("Real value found: %f", val_1)
                values[col_i] = val_1
                col_i += 1
        else:
            logging.debug("No values found, improving...")
            Q, R = qr_decomposition(matrix)
            matrix = R @ Q

    if values[-1] == 0:
        values[-1] = matrix[-1, -1]

    logging.debug(f"Resulting matrix:\n{matrix}")
    
    return values

In [7]:
eigens = qr_algorithm(matrix, eps)
print("Eigenvalues: ")
print(eigens)

DEBUG:root:Reduced matrix:
[[10.027277   -1.28576762 -2.55815559]
 [-0.00034816  2.51217802  7.78368006]
 [-0.00007825 -5.05391852  5.46054498]]
DEBUG:root:No values found, improving...
DEBUG:root:No values found, improving...
DEBUG:root:No values found, improving...
DEBUG:root:No values found, improving...
DEBUG:root:Real value found: 10.027252
DEBUG:root:Complex values found: (-3.9863732424333262+6.09624814668469j) (-3.9863732424333262-6.09624814668469j)
DEBUG:root:Resulting matrix:
[[10.02725205 -1.15957511  2.61806351]
 [-0.00007501  2.9320109   4.70862117]
 [-0.00002763 -8.12886917  5.04073705]]


Eigenvalues: 
[10.027252 +0.j       -3.9863732+6.096248j -3.9863732-6.096248j]


Проверить можно исходя из ортогональности собственных векторов друг другу:

Видно, что в пределах погрешности они равны нулю.